<a href="https://colab.research.google.com/github/sibat119/pytorch-intro/blob/main/linear_regression_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

In [9]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [10]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [11]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [22]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w, b)

tensor([[ 1.4825,  0.3620, -0.2856],
        [-1.7643,  0.1172,  0.0777]], requires_grad=True) tensor([-1.2507, -0.8132], requires_grad=True)


In [23]:
def model(x):
  return x @ w.t() + b

In [24]:
preds = model(inputs)
print(preds)

tensor([[ 118.9490, -118.4122],
        [ 147.2397, -146.0762],
        [ 159.6741, -134.0942],
        [ 154.9684, -172.8558],
        [ 115.8063, -105.8578]], grad_fn=<AddBackward0>)


In [25]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [26]:
loss = mse(targets, preds)
learning_rate = 1e-4
print(loss)

tensor(29033.4844, grad_fn=<DivBackward0>)


In [28]:
loss.backward()
print(w)
print(w.grad)

tensor([[ 1.4825,  0.3620, -0.2856],
        [-1.7643,  0.1172,  0.0777]], requires_grad=True)
tensor([[  5721.6284,   4488.8086,   3024.3027],
        [-19279.1387, -20153.4180, -12582.1553]])


In [30]:
with torch.no_grad():
  w -= w.grad * learning_rate
  b -= b.grad * learning_rate
  w.grad.zero_()
  b.grad.zero_()

In [31]:
print(w)
print(b)

tensor([[ 0.3382, -0.5358, -0.8904],
        [ 2.0915,  4.1479,  2.5942]], requires_grad=True)
tensor([-1.2633, -0.7678], requires_grad=True)


In [32]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(186440.4062, grad_fn=<DivBackward0>)


In [34]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * learning_rate
        b -= b.grad * learning_rate
        w.grad.zero_()
        b.grad.zero_()

In [35]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(7.9479, grad_fn=<DivBackward0>)


In [36]:
preds

tensor([[ 57.2834,  70.2291],
        [ 79.6609, 100.2759],
        [124.2895, 133.9812],
        [ 22.7301,  37.2913],
        [ 96.2804, 118.1337]], grad_fn=<AddBackward0>)

In [37]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins
We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks

In [38]:
import torch.nn as nn

In [39]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [40]:
from torch.utils.data import TensorDataset

In [41]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [42]:
from torch.utils.data import DataLoader

In [43]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [44]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  97.,  70.],
        [ 68.,  96.,  71.],
        [ 88., 134.,  59.],
        [ 87., 135.,  57.],
        [ 73.,  66.,  44.]])
tensor([[102., 120.],
        [104., 118.],
        [118., 132.],
        [118., 134.],
        [ 57.,  69.]])


In [45]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1143, -0.5644,  0.0871],
        [-0.5544,  0.0777, -0.0666]], requires_grad=True)
Parameter containing:
tensor([-0.2024, -0.0007], requires_grad=True)


In [46]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1143, -0.5644,  0.0871],
         [-0.5544,  0.0777, -0.0666]], requires_grad=True),
 Parameter containing:
 tensor([-0.2024, -0.0007], requires_grad=True)]

In [47]:
?torch.utils.data.DataLoader

In [49]:
?torch.nn.Linear

In [50]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-25.9255, -38.1330],
        [-33.8907, -47.8802],
        [-60.8316, -41.6885],
        [ -9.5883, -55.6764],
        [-40.3978, -35.4610],
        [-25.2468, -38.7651],
        [-33.2392, -48.0245],
        [-60.6302, -42.3095],
        [-10.2670, -55.0442],
        [-40.4250, -34.9732],
        [-25.2741, -38.2773],
        [-33.2120, -48.5123],
        [-61.4831, -41.5441],
        [ -9.5611, -56.1642],
        [-41.0765, -34.8289]], grad_fn=<AddmmBackward>)

In [51]:
# Import nn.functional
import torch.nn.functional as F
# Define loss function
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(17044.9629, grad_fn=<MseLossBackward>)


In [52]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [53]:
?torch.optim.SGD

In [54]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [55]:
fit(150, model, loss_fn, opt, train_dl)

Epoch [10/150], Loss: 406.6706
Epoch [20/150], Loss: 266.7515
Epoch [30/150], Loss: 253.4786
Epoch [40/150], Loss: 179.9372
Epoch [50/150], Loss: 104.1842
Epoch [60/150], Loss: 69.8516
Epoch [70/150], Loss: 35.4035
Epoch [80/150], Loss: 37.2124
Epoch [90/150], Loss: 59.4601
Epoch [100/150], Loss: 15.4403
Epoch [110/150], Loss: 23.1780
Epoch [120/150], Loss: 18.2408
Epoch [130/150], Loss: 13.8711
Epoch [140/150], Loss: 11.3442
Epoch [150/150], Loss: 14.8866


In [56]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.5943,  70.7947],
        [ 81.8388,  98.2188],
        [117.7418, 137.3202],
        [ 24.5785,  39.8957],
        [ 99.2788, 113.2248],
        [ 56.4374,  69.6903],
        [ 81.6112,  97.8799],
        [118.0189, 137.6909],
        [ 25.7354,  41.0001],
        [100.2081, 113.9902],
        [ 57.3667,  70.4558],
        [ 80.6819,  97.1145],
        [117.9695, 137.6591],
        [ 23.6492,  39.1303],
        [100.4357, 114.3291]], grad_fn=<AddmmBackward>)

In [57]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])